# Creating raw setinments data to the tabluer form for batter processing

In [1]:
#importing usefull libraies
import csv,unicodedata,sys

In [2]:
# importing sentences text file to the form of list and also performe some miner pre processing
sentences={}
with open("stanfordSentimentTreebank/datasetSentences.txt","r") as f:
	rd=csv.reader(f,delimiter='\t')
	count=0
	for line in rd:
		if count==0:
			count=1
			continue
		line[1]=line[1].replace('-LRB-','(') # replecing some widly encoded textto there actuly values manully 
		line[1]=line[1].replace('-RRB-',')')
		line[1]=line[1].replace('Â', '')
		line[1]=line[1].replace('Ã©', 'e')
		line[1]=line[1].replace('Ã¨', 'e')
		line[1]=line[1].replace('Ã¯', 'i')
		line[1]=line[1].replace('Ã³', 'o')
		line[1]=line[1].replace('Ã´', 'o')
		line[1]=line[1].replace('Ã¶', 'o')
		line[1]=line[1].replace('Ã±', 'n')
		line[1]=line[1].replace('Ã¡', 'a')
		line[1]=line[1].replace('Ã¢', 'a')
		line[1]=line[1].replace('Ã£', 'a')
		line[1]=line[1].replace('\xc3\x83\xc2\xa0', 'a')
		line[1]=line[1].replace('Ã¼', 'u')
		line[1]=line[1].replace('Ã»', 'u')
		line[1]=line[1].replace('Ã§', 'c')
		line[1]=line[1].replace('Ã¦', 'ae')
		line[1]=line[1].replace('Ã­', 'i')
		line[1]=line[1].replace('\xa0', ' ')
		line[1]=line[1].replace('\xc2', '')
		sentences[line[0]]=line[1]

In [3]:
# importing data split text and split the values accourding to traing,testing and validation data
train={}
test={}
dev={}
sents=[]
with open("stanfordSentimentTreebank/datasetSplit.txt","r") as f:
	rd=csv.reader(f,delimiter=',')
	count=0
	for line in rd:
		if count==0:
			count=1
			continue
		if line[1]=='1':
			train[sentences[line[0]]]=0
			sents.append(sentences[line[0]])
		elif line[1]=='2':
			test[sentences[line[0]]]=0
		elif line[1]=='3':
			dev[sentences[line[0]]]=0

In [4]:
%%time
# importing dictionery and match there recpective sentences from the dictionery
train_sent = train.copy()
string=" ".join(sents)
with open("stanfordSentimentTreebank/dictionary.txt","r") as f:
	rd=csv.reader(f,delimiter='|')
	for line in rd:
		line[0]=line[0].replace('é','e')
		line[0]=line[0].replace('è','e')
		line[0]=line[0].replace('ï','i')
		line[0]=line[0].replace('í','i')
		line[0]=line[0].replace('ó','o')
		line[0]=line[0].replace('ô','o')
		line[0]=line[0].replace('ö','o')
		line[0]=line[0].replace('á','a')
		line[0]=line[0].replace('â','a')
		line[0]=line[0].replace('ã','a')
		line[0]=line[0].replace('à','a')
		line[0]=line[0].replace('ü','u')
		line[0]=line[0].replace('û','u')
		line[0]=line[0].replace('ñ','n')
		line[0]=line[0].replace('ç','c')
		line[0]=line[0].replace('æ','ae')
		line[0]=line[0].replace('\xa0', ' ')
		line[0]=line[0].replace('\xc2', '')
		if line[0] in string:
			train[line[0]]=line[1]
		if line[0] in test:
			test[line[0]]=line[1]
		if line[0] in train_sent:
			train_sent[line[0]]=line[1]
		if line[0] in dev:
			dev[line[0]]=line[1]

Wall time: 1min 7s


In [5]:
# importing the sentiment lables and matching with there sentences IDs
labels={}
with open("stanfordSentimentTreebank/sentiment_labels.txt","r") as f:
	rd=csv.reader(f,delimiter='|')
	count=0
	for line in rd:
		if count==0:
			count=1
			continue
		labels[line[0]]=float(line[1])

In [6]:
# replcening 0(integer) sentences ID to the string(0) sentences ids
for key, value in train.items():
    if value == 0:
        train[key] = "0"
for key, value in train_sent.items():
    if value == 0:
        train_sent[key] = "0"
for key, value in test.items():
    if value == 0:
        test[key] = "0"
for key, value in dev.items():
    if value == 0:
        dev[key] = "0"

In [7]:
# putting sentiments score accourding to there sentences Ids
for key in train:
    train[key]=labels[train[key]]
for key in train_sent:
	train_sent[key]=labels[train_sent[key]]
for key in test:
	test[key]=labels[test[key]]
for key in dev:
	dev[key]=labels[dev[key]]

In [8]:
print(len(train))
print(len(train_sent))
print(len(test))
print(len(dev))

169477
8534
2210
1100


In [9]:
# converting list to the dataframe
import pandas as pd
train = pd.DataFrame(train.items(),columns=['Sentence', 'Sentiment_Values'])
train_sent = pd.DataFrame(train_sent.items(),columns=['Sentence', 'Sentiment_Values'])
test = pd.DataFrame(test.items(),columns=['Sentence', 'Sentiment_Values'])
dev = pd.DataFrame(dev.items(),columns=['Sentence', 'Sentiment_Values'])

In [10]:
df = pd.concat([train,train_sent,test,dev])

In [11]:
df

,Sentence,Sentiment_Values
0,The Rock is destined to be the 21st Century 's...,0.69444
1,The gorgeously elaborate continuation of `` Th...,0.83333
2,Singer\/composer Bryan Adams contributes a sle...,0.62500
3,You 'd think by now America would have had eno...,0.50000
4,Yet the act is still charming here .,0.72222
...,...,...
1095,it seems to me the film is about the art of ri...,0.29167
1096,It 's just disappointingly superficial -- a mo...,0.33333
1097,The title not only describes its main characte...,0.23611
1098,Sometimes it feels as if it might have been ma...,0.44444


In [12]:
#converting sentiment values to the float
df = df.astype({'Sentiment_Values': 'float64'})

In [13]:
df.dtypes

Sentence             object
Sentiment_Values    float64
dtype: object

In [14]:
# Create a function to provieds sentimnets accourding to there sentiments values. 
# this are created by instructs provided in a readme file.
def getAnalysis(x):
    if x<= 0.2:
        return 'Very Negative'
    elif x<=0.4:
        return 'Negative'
    elif x<=0.6:
        return 'Netural'
    elif x<=0.8:
        return 'Positive'
    elif x<=1:
        return 'Very Positive'

In [15]:
df['Sentiment'] = df['Sentiment_Values'].apply(getAnalysis)

In [16]:
df = df.reset_index(drop=True)

In [17]:
df

,Sentence,Sentiment_Values,Sentiment
0,The Rock is destined to be the 21st Century 's...,0.69444,Positive
1,The gorgeously elaborate continuation of `` Th...,0.83333,Very Positive
2,Singer\/composer Bryan Adams contributes a sle...,0.62500,Positive
3,You 'd think by now America would have had eno...,0.50000,Netural
4,Yet the act is still charming here .,0.72222,Positive
...,...,...,...
181316,it seems to me the film is about the art of ri...,0.29167,Negative
181317,It 's just disappointingly superficial -- a mo...,0.33333,Negative
181318,The title not only describes its main characte...,0.23611,Negative
181319,Sometimes it feels as if it might have been ma...,0.44444,Netural


In [18]:
# saving finel data in a tabulr form in CSV file.
df.to_csv (r'Sentiment_data.csv', index = None, header=True) 